<a href="https://colab.research.google.com/github/khadkechetan/information_extraction/blob/main/LLM/Information_extraction_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install all the packages

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index pypdf python-dotenv

# 2. Packages imports

In [ ]:
import logging
import sys
import torch
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.prompts.prompts import SimpleInputPrompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# 3. Login to HuggingFace cli

In [ ]:
!git config --global credential.helper store
!huggingface-cli login

# 4. Load the Dataset

In [ ]:
data_location = '/content/data' #@param
documents = SimpleDirectoryReader(data_location).load_data()

In [ ]:
documents[2]

In [ ]:
system_prompt = "You are a data extractor. Extract the data from given document"
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

# 5. Load the Embeddings

In [ ]:
# hyperparameters
context_window = 4096 #@param
temperature = 0.0 #@param
model_name = 'meta-llama/Llama-2-7b-chat-hf' #@param

In [ ]:
llm = HuggingFaceLLM(
    context_window=context_window,
    max_new_tokens=256,
    generate_kwargs={"temperature": temperature, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True, "use_auth_token":True},

)

In [ ]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

#6. Index the datastore

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
user_query = 'What is Date issued in the document?'  #@param

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is Date issued in the document?")

In [ ]:
print(response)

The purpose of the loan in the document is to provide financing for the purchase of a property located at 456 Somewhere Ave., Anytown, ST 12345. The loan amount is $162,000, and the loan term is 30 years. The interest rate is 4.174%, and the total amount of interest paid over the loan term is $69,460. The loan also includes an escrow account for property taxes and homeowner's insurance.
